## Test AI Angent Work flow for Bank Statements

In [158]:
import os, json, time, gc

from dotenv import load_dotenv
from IPython.display import HTML, Markdown, Image, Video
from tqdm import tqdm
from openai import OpenAI, AsyncOpenAI
import asyncio
import aiohttp
import pandas as pd
import torch
from ctypes import *

import fitz # PyMuPDF is imported as 'fitz' 

#fix bug with aysncio and jupyter
import nest_asyncio # for langchain async 
nest_asyncio.apply()

import numpy as np


from litellm.caching import Cache

# Create a Cache object
litellm.cache = Cache()


# load environment variables from .env file
load_dotenv()

True

In [159]:
class CFG:
    


    USE_OPIK = True #False
    OFFLINE = False #True # for Test offline environment
    USE_LLAMA3 = False # 
    USE_GEMMA2 = False # 
    USE_QWEN = False # 
    USE_DEEPSEEK = True # 
    USE_DEEPSCALE = False # 

    TASK_GEN = True # for generative Text output task (suitable for RAG project)
    TEST_LLM = True
    # Set LLM host framework
    USE_OLLAMA = True # for OLLAMA local LLM framework
    USE_OPENROUTER = False # for OpenRouter LLM framework
    USE_OPENAI = False # for OpenAI LLM framework
    USE_HUGGINGFACE = True # Pull model from Huggingface model hub
    USE_LMSTUIDO = False # for local LLM framework  
    USE_VLLM = False # for VLLM  LLM framework

    # mulitlingual LLM model 
    model1 = "meta-llama/Llama-3.2-3B-Instruct"  # llama3.2  3B-Instruct

    model2 =  "google/gemma-2-2b-it" # gemma 2 9B (mulitlingual)
    model3 = "Qwen/Qwen2.5-3B-Instruct" # Qwen 3B (mulitlingual)
    model4 = 'Qwen/Qwen2.5-7B-Instruct' # Qwen 7B (mulitlingual)
    model5 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" # DeepSeek Distill 1.5B (mulitlingual)
    model6 = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B" # DeepSeek Distill 7B (mulitlingual)
    model7 = "agentica-org/DeepScaleR-1.5B-Preview"

    # for VLM model
    vlmModel1 = "Qwen/Qwen2.5-VL-3B-Instruct"
    vlmModel2 = "Qwen/Qwen2.5-VL-7B-Instruct"


    # Mult Embedding model
    embedModel1 = "intfloat/multilingual-e5-small-instruct" 
    embedModel2 = "intfloat/multilingual-e5-large-instruct"
    embedModel3 = "Alibaba-NLP/gte-Qwen2-1.5B-instruct" # for embedding model support chinese
    embedModel4 = "Alibaba-NLP/gte-multilingual-base" # for embedding model support chinese
    embedModel5 = "BAAI/bge-m3" # for multilingual embedding model
    embedModel6 = "jinaai/jina-embeddings-v3"
    embedModel7 = "ollama/mxbai-embed-large"  # ollama 
    embedModel8 = "ollama/nomic-embed-text"  # ollama 
    

    # LLM for AI Agent  OLLAMA (for local deployment) ,requires pulling the ollama image and running the server locally
    llmModel1 = "ollama/deepseek-r1:8b"
    llmModel2 = "ollama/deepseek-r1:7b"
    llmModel3 = "ollama/qwen2.5" # Qwen 2.5 7B
    llmModel4 = "ollama/llama3.2"
    llm_base_url1 = "http://localhost:11434"

    #LLM for AI Agent Openrouter 
    llmModel5 = "openrouter/qwen/qwen3-32b:free" # for openrouter
    llmModel6 = "openrouter/google/gemini-2.0-flash-exp:free"
    llmModel7 = "ollama/qwen3:8b" # Qwen 3.2 8B
    llm_base_url2="https://openrouter.ai/api/v1"
    api_key2 = os.getenv("OPENROUTER_API_KEY")
   
    # 



    FEW_SHOT_TEST= False#True
    USE_WANDB = True#True # for  LLM evalution and debug , track fine tuning performance

    USE_DEEPEVAL = True#False # for LLM evalution   
    USE_TRAIN =  False #True #False#True Much be use GPU for Training 
    
    # For VectorDB selection
    USE_FAISS = False#True # For RAG VectorDB
    USE_CHROMA = True #False #True #False # for RAG VectorDF
    USE_PINECONE = False#True#False #True # for RAG VectorDF
    USE_WEAVIATE = False#True #False # for RAG VectorDF
    USE_MILVUS = False#True              # for RAG VectorDF

    # for LLM fine tuning
    maxTrainData = 200#3500#5000 #10000#5000 #10000
    maxEvalData = 20#100 # 20 


    # LLM parameters
    reportTo ="none"
    topK = 40
    topP = 1.0
    temperature = 0.6 #0.5
    repetition_penalty = 1.05 # 1.1
    maxOutToken = 1024#180 #100
    

    
    maxToken=  512#768#512#768 # 512 for test only

In [160]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai.tools import BaseTool, tool
from crewai import LLM
import crewai
from pydantic import BaseModel, Field
from typing import List, Dict, Type
from crewai_tools import PDFSearchTool

In [161]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [162]:
from langchain_ollama import OllamaLLM, ChatOllama
from langchain_openai import ChatOpenAI , OpenAIEmbeddings
from openai import OpenAI, AsyncOpenAI


In [163]:
import ctypes
def clearMemory():
    for _ in range(5):
        torch.cuda.empty_cache()
        ctypes.CDLL("libc.so.6").malloc_trim(0)
        gc.collect()
        time.sleep(0.3)

In [164]:
# clearMemory()

In [165]:
# !pip3 install --upgrade pip
# !pip3 install -qU langchain-qdrant
# !pip3 install -qU langchain-ollama

In [166]:
## Define Test Document path 
pdfFilePath1 = "../test-document/Attention .pdf"
pdfFilePath2 = "../test-document/yolo.pdf"
pdfDir = "../test-document"

bankStatementDir = "../bank-statement-document/"
bankStatementSamples =  "../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf"

## CrewAI Support LLM List :  LiteLLM 
- <https://docs.crewai.com/how-to/llm-connections>
- <https://docs.crewai.com/concepts/llms>



In [167]:
from crewai_tools import PDFSearchTool, YoutubeChannelSearchTool
from crewai_tools import RagTool

from crewai.tools import BaseTool, tool

# Chanin
from langchain_community.document_loaders import (TextLoader,
                                                  PyMuPDFLoader,
                                                  PyPDFDirectoryLoader,
                                                  PyPDFLoader)
from langchain_core.documents import Document

In [168]:
# # for ML Model
# if CFG.USE_WANDB:
#     # train report to  W&B tool
#     # import wandb
#     # reportTo= "wandb"
#     # my_secret = os.getenv("wandb_api_key") 
#     # wandb.login(key=my_secret) # login 
#     # wandb.init(
# else: 
#     reportTo = "none"# None

if CFG.USE_WANDB:
    import wandb
    # reportTo= "wandb"
    my_secret = os.getenv("wandb_api_key") 
    wandb.login(key=my_secret) # login 
    import weave
    # Initialize Weave with your project name 
    weave.init("crewai-ai-bank-statement-document-analysis-project")




wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/johnsonhk88/.netrc


In [169]:
import opik
from opik import track
from opik.integrations.crewai import track_crewai
# for LiteLLM in crew 
from litellm.integrations.opik.opik import OpikLogger
import litellm

if CFG.USE_OPIK:
    opik.configure(use_local=True , automatic_approvals=True , url="http://localhost:5173")

    track_crewai(project_name="crewai-ai-bank-statement-document-analysis-project")
    opik_logger = OpikLogger()
    litellm.callbacks = [opik_logger]

OPIK: Existing Opik clients will not use updated values for "url", "api_key", "workspace".


### test pdf document loader

In [170]:


loader = PyMuPDFLoader(bankStatementSamples, extract_tables="markdown")
documents = loader.load()
docs= []
for doc in documents:
    # print(doc)
    print("=====================================")
    print(f"page content: {doc.page_content}")
    print(f"metaData : {doc.metadata}")
    print("=====================================")
    # print
    temp= {
        "page_content": doc.page_content,
        "metadata": doc.metadata

    }
    docs.append(temp)
df = pd.DataFrame(docs , columns=["page_content", "metadata"])

page content: Issue Date:
Period:
Account Activity
Date
Payment Type
Paid In
Paid Out
Balance
Your Account Statement
Detail
Note:
Print Form
Save Form
Reset Form
<Branch Name>
231 Valley Farms Street 
Santa Monica, CA 
bickslowbank@domain.com
mm/dd/yyyy
mm/dd/yyyy to mm/dd/yyyy
111-234-567-890  
Bit Manufacturing Ltd
2450 Courage St, STE 108
Brownsville, TX 78521
Balance Brought Forward
8,313.30
mm/dd/yyyy Fast Payment
Amazon
132.30
8,181.00
mm/dd/yyyy BACS
eBAY Trading Co.
515.22
7,665.78
mm/dd/yyyy Fast Payment
Morrisons Petrol
80.00
7,585.78
mm/dd/yyyy BACS
Business Loan
20,000.00
27,585.78
mm/dd/yyyy BACS
Jumes White Media
2,416.85
25,168.93
mm/dd/yyyy Fast Payment
ATM High Street
100.00
25,068.93
mm/dd/yyyy BACS
Accorn Advertising Studios
150.00
24,918.93
Fast Payment
mm/dd/yyyy
Marriott Hotels
177.00
24,741.93
mm/dd/yyyy Fast Payment
Abelio Scotrail Ltd
122.22
24,619.71
mm/dd/yyyy Fast Payment
Cheque 000234
1,200.00
23,419.71
mm/dd/yyyy Int. Bank
Interest Paid
9.33
23,429.04
mm/d

In [171]:
# !pip3 list

In [172]:
# !pip freeze > new-requirements.txt

In [173]:
# df

In [174]:
tempDict = df.to_dict(orient="records")
tempDict[0]

{'page_content': 'Issue Date:\nPeriod:\nAccount Activity\nDate\nPayment Type\nPaid In\nPaid Out\nBalance\nYour Account Statement\nDetail\nNote:\nPrint Form\nSave Form\nReset Form\n<Branch Name>\n231 Valley Farms Street \nSanta Monica, CA \nbickslowbank@domain.com\nmm/dd/yyyy\nmm/dd/yyyy to mm/dd/yyyy\n111-234-567-890  \nBit Manufacturing Ltd\n2450 Courage St, STE 108\nBrownsville, TX 78521\nBalance Brought Forward\n8,313.30\nmm/dd/yyyy Fast Payment\nAmazon\n132.30\n8,181.00\nmm/dd/yyyy BACS\neBAY Trading Co.\n515.22\n7,665.78\nmm/dd/yyyy Fast Payment\nMorrisons Petrol\n80.00\n7,585.78\nmm/dd/yyyy BACS\nBusiness Loan\n20,000.00\n27,585.78\nmm/dd/yyyy BACS\nJumes White Media\n2,416.85\n25,168.93\nmm/dd/yyyy Fast Payment\nATM High Street\n100.00\n25,068.93\nmm/dd/yyyy BACS\nAccorn Advertising Studios\n150.00\n24,918.93\nFast Payment\nmm/dd/yyyy\nMarriott Hotels\n177.00\n24,741.93\nmm/dd/yyyy Fast Payment\nAbelio Scotrail Ltd\n122.22\n24,619.71\nmm/dd/yyyy Fast Payment\nCheque 000234\n1,20

In [175]:
#
def convertDataFrameToDocs(df):
    """ Convert DataFrame to list of LangChain Documents"""
    docs = []
    tempDict = df.to_dict(orient="records")
    for temp in tempDict:
        document = Document(
            page_content=temp["page_content"],
            metadata=temp["metadata"]
        )
        docs.append(document)
    # document = Document(
    #     page_content=tempDict[0]["page_content"],
    #     metadata=tempDict[0]["metadata"]
    # )
    return docs

In [176]:
# Convert DataFrame to LangChain Document
pdfDocs = convertDataFrameToDocs(df)

In [177]:
# save the document to Vector Database
pdfDocs

[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2020-07-03T16:22:11+08:00', 'source': '../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf', 'file_path': '../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': 'Bank Statement Template 2 - TemplateLab.xlsx', 'author': 'HFO Desktop', 'subject': '', 'keywords': '', 'moddate': '2020-07-03T19:36:51+08:00', 'trapped': '', 'modDate': "D:20200703193651+08'00'", 'creationDate': "D:20200703162211+08'00'", 'page': 0}, page_content='Issue Date:\nPeriod:\nAccount Activity\nDate\nPayment Type\nPaid In\nPaid Out\nBalance\nYour Account Statement\nDetail\nNote:\nPrint Form\nSave Form\nReset Form\n<Branch Name>\n231 Valley Farms Street \nSanta Monica, CA \nbickslowbank@domain.com\nmm/dd/yyyy\nmm/dd/yyyy to mm/dd/yyyy\n111-234-567-890  \nBit Manufacturing Ltd\n2450 Courage St, STE 108\nBrownsville, TX 78521\nBalance Brought Forward\

# Embedding Model

In [178]:
# from langchain.vectorstores import Qdrant
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

from sentence_transformers import SentenceTransformer 

from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma #
from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings
from openai import OpenAI
from langchain_ollama import OllamaEmbeddings

# 
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [179]:
def HuggingFaceEmbeddingInit(modelName: str= CFG.embedModel1 , remote_code: bool = False):
    """ Initialize the HuggingFace Embedding model"""
    if device.type =="cpu":
        # model_kwargs = {'device': 'cpu', "torch_dtype": "float16"} # not support torch_dtype in Hugggiungface embeding at langchain
        model_kwargs = {'device': 'cpu', "trust_remote_code": remote_code}
    else :
        model_kwargs = {"device": "cuda", "trust_remote_code": remote_code}
    embedding_model = HuggingFaceEmbeddings(
        model_name=modelName,
        model_kwargs=model_kwargs
    )
    return embedding_model


def OllamaEmbeddingInit(modelName: str =CFG.embedModel8 , url=None):
    """ Initialize the Ollama Embedding model"""
    embedding_model = OllamaEmbeddings(
        model=modelName,
        base_url=url
    )
    return embedding_model

In [180]:
# Initialize the HuggingFace Embedding model
# embed  =HuggingFaceEmbeddingInit(modelName=CFG.embedModel3,  remote_code=True)
embed = OllamaEmbeddingInit(modelName="nomic-embed-text", url=None)
embed

OllamaEmbeddings(model='nomic-embed-text', base_url=None, client_kwargs={}, mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, keep_alive=None, num_thread=None, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None)

In [181]:
# # Initialize Qdrant vector database
# qdrant = Qdrant.from_documents(
#     documents=[pdfDocs],  # List of LangChain documents
#     embedding=embed,
#     location=":memory:",  # Use in-memory Qdrant instance; replace with actual Qdrant URL if needed
#     collection_name="bank_statements"
# )

# print("Document saved to Qdrant vector database.")

def initQdrantDB(embedding_model, collection_name: str = "bank_statements", embedding_dimension: int = 768):
    """ Initialize Qdrant vector database"""
    client = QdrantClient(":memory:")
    
    # Check if the collection already 
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=embedding_dimension, distance=Distance.COSINE),
    )

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embedding_model,
    )
    # qdrant = QdrantClient(url=url)
    # qdrant.recreate_collection(
    #     collection_name=collection_name,
    #     vectors_config=VectorParams(
    #         size=embedding_model.embedding_dimension,
    #         distance=Distance.COSINE
    #     )
    # )
    return vector_store


def saveToVDB(db, chuck: list[Document]):
    startTime = time.time()
    ids = db.add_documents(chuck)
    print(f"Time Taken:  {time.time() - startTime}")
    return ids    


async def saveToVDBAsync(db, chuck: list[Document]):
    startTime = time.time()
    ids = await db.aadd_documents(chuck)
    print(f"Time Taken:  {time.time() - startTime}")
    return ids    


def vectorDBsimilaritySearchWithScores(db, query, k =3):
        results = db.similarity_search_with_relevance_scores(query, k=k)
        return results

def vectorDBsimilaritySearch(db, query, k=3):
        results = db.similarity_search(query, k=k)
        return results


def vectorDBsimilaritySearchByVector(db, query, embed, k=3 ):
        results = db.similarity_search_by_vector(
        embedding=embed.embed_query(query), k=k)
        return results

def vectorDBSimilaritySearchByMMR(db, query, k=3, fetchK=20):
        results = db.max_marginal_relevance_search(query=query, k=k, fetch_k=fetchK)
        return results


In [182]:



def textSplitterByText(txt, chunkSize= 512, overlap= 20, separators=["\n\n"]):
    textSplitter = RecursiveCharacterTextSplitter(
        chunk_size = chunkSize,
        chunk_overlap = overlap,
        separators=separators,
        length_function = len,
        is_separator_regex =False
    )
    splittedTxt = textSplitter.split_text(txt)
    return splittedTxt

def textSplitterByDocs(docs, chunkSize= 512, overlap= 20, separators=["\n\n"]):
    textSplitter = RecursiveCharacterTextSplitter(
        chunk_size = chunkSize,
        chunk_overlap = overlap,
        separators=separators,
        length_function = len,
        is_separator_regex =False
    )
    splitted = textSplitter.split_documents(docs)
    return splitted


In [183]:
vectDB = initQdrantDB(embed, collection_name="bank_statements", embedding_dimension=768)

In [184]:
splittedDocs = textSplitterByDocs(pdfDocs, chunkSize=512, overlap=20, separators=["\n\n"])

In [185]:
# splittedDocs[].metadata

In [186]:
ids = await saveToVDBAsync(vectDB, splittedDocs)

Time Taken:  0.031774282455444336


In [187]:
ids

['103616660a3b4f81810b4e976d84645f']

In [188]:
### Test Data Retrieval from Vector Database

In [189]:
res =vectDB.similarity_search("What is the total amount in the bank statement?", k=3)

In [190]:
res , len(res)

([Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2020-07-03T16:22:11+08:00', 'source': '../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf', 'file_path': '../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': 'Bank Statement Template 2 - TemplateLab.xlsx', 'author': 'HFO Desktop', 'subject': '', 'keywords': '', 'moddate': '2020-07-03T19:36:51+08:00', 'trapped': '', 'modDate': "D:20200703193651+08'00'", 'creationDate': "D:20200703162211+08'00'", 'page': 0, '_id': '103616660a3b4f81810b4e976d84645f', '_collection_name': 'bank_statements'}, page_content='Issue Date:\nPeriod:\nAccount Activity\nDate\nPayment Type\nPaid In\nPaid Out\nBalance\nYour Account Statement\nDetail\nNote:\nPrint Form\nSave Form\nReset Form\n<Branch Name>\n231 Valley Farms Street \nSanta Monica, CA \nbickslowbank@domain.com\nmm/dd/yyyy\nmm/dd/yyyy to mm/dd/yyyy\n111-234-567-890  \nBit Manufactu

In [191]:
for val in res:
    print(f"* {val.page_content}\nMetaData[{val.metadata}]")
    print("=====================================")

* Issue Date:
Period:
Account Activity
Date
Payment Type
Paid In
Paid Out
Balance
Your Account Statement
Detail
Note:
Print Form
Save Form
Reset Form
<Branch Name>
231 Valley Farms Street 
Santa Monica, CA 
bickslowbank@domain.com
mm/dd/yyyy
mm/dd/yyyy to mm/dd/yyyy
111-234-567-890  
Bit Manufacturing Ltd
2450 Courage St, STE 108
Brownsville, TX 78521
Balance Brought Forward
8,313.30
mm/dd/yyyy Fast Payment
Amazon
132.30
8,181.00
mm/dd/yyyy BACS
eBAY Trading Co.
515.22
7,665.78
mm/dd/yyyy Fast Payment
Morrisons Petrol
80.00
7,585.78
mm/dd/yyyy BACS
Business Loan
20,000.00
27,585.78
mm/dd/yyyy BACS
Jumes White Media
2,416.85
25,168.93
mm/dd/yyyy Fast Payment
ATM High Street
100.00
25,068.93
mm/dd/yyyy BACS
Accorn Advertising Studios
150.00
24,918.93
Fast Payment
mm/dd/yyyy
Marriott Hotels
177.00
24,741.93
mm/dd/yyyy Fast Payment
Abelio Scotrail Ltd
122.22
24,619.71
mm/dd/yyyy Fast Payment
Cheque 000234
1,200.00
23,419.71
mm/dd/yyyy Int. Bank
Interest Paid
9.33
23,429.04
mm/dd/yyyy DD
OV

In [212]:
print(df.iloc[0]["metadata"])

{'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2020-07-03T16:22:11+08:00', 'source': '../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf', 'file_path': '../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': 'Bank Statement Template 2 - TemplateLab.xlsx', 'author': 'HFO Desktop', 'subject': '', 'keywords': '', 'moddate': '2020-07-03T19:36:51+08:00', 'trapped': '', 'modDate': "D:20200703193651+08'00'", 'creationDate': "D:20200703162211+08'00'", 'page': 0}


In [213]:
documents[0].page_content

'Issue Date:\nPeriod:\nAccount Activity\nDate\nPayment Type\nPaid In\nPaid Out\nBalance\nYour Account Statement\nDetail\nNote:\nPrint Form\nSave Form\nReset Form\n<Branch Name>\n231 Valley Farms Street \nSanta Monica, CA \nbickslowbank@domain.com\nmm/dd/yyyy\nmm/dd/yyyy to mm/dd/yyyy\n111-234-567-890  \nBit Manufacturing Ltd\n2450 Courage St, STE 108\nBrownsville, TX 78521\nBalance Brought Forward\n8,313.30\nmm/dd/yyyy Fast Payment\nAmazon\n132.30\n8,181.00\nmm/dd/yyyy BACS\neBAY Trading Co.\n515.22\n7,665.78\nmm/dd/yyyy Fast Payment\nMorrisons Petrol\n80.00\n7,585.78\nmm/dd/yyyy BACS\nBusiness Loan\n20,000.00\n27,585.78\nmm/dd/yyyy BACS\nJumes White Media\n2,416.85\n25,168.93\nmm/dd/yyyy Fast Payment\nATM High Street\n100.00\n25,068.93\nmm/dd/yyyy BACS\nAccorn Advertising Studios\n150.00\n24,918.93\nFast Payment\nmm/dd/yyyy\nMarriott Hotels\n177.00\n24,741.93\nmm/dd/yyyy Fast Payment\nAbelio Scotrail Ltd\n122.22\n24,619.71\nmm/dd/yyyy Fast Payment\nCheque 000234\n1,200.00\n23,419.71\n

# Define the tools for CrewAI agent

In [214]:
#Tool for AI agent

# pdfTool = PDFSearchTool(pdfDir=bankStatementDir)

In [215]:
# Custom PDF Search Tool for CrewAI
@tool("Custom PDF Extractor")
def pdf_extractor(
    pdf_dir: str
) -> Dict:
    """
    Extracts text from PDF files in the specified directory.
    Args:
        pdf_dir (str): The directory containing PDF files.
    Returns:
        List[Dict[str, str]]: A list of dictionaries containing the extracted text from each PDF file.
    """
    # results = pdfTool.search(query=query, pdf_dir=pdf_dir)
    loader = PyMuPDFLoader(pdf_dir)
    documents = loader.load() #return list of documents
    docs = []
    for doc in documents:
        # print(doc)
        print("=====================================")
        print(f"page content: {doc.page_content}")
        print(f"metaData : {doc.metadata}")
        print("=====================================")
        # print
        temp= {
            "page_content": doc.page_content,
            "metadata": doc.metadata

        }
        docs.append(temp)
    
    df = pd.DataFrame(docs , columns=["page_content", "metadata"])
    print(f"Extracted {len(df)} pages from PDF files in {pdf_dir}\n\r {type(df)}, {df}")
    return df


@tool("PDF directory Extractor")
def pdf_directory_extractor(
    pdf_dir: str = pdfDir
) -> list:
    """
    Extracts text from all PDF files in the specified directory.
    Parameters:
    - pdf_dir (str): The directory containing PDF files.
    Returns:
    - List of dictionaries containing the content and source of each PDF file.
    """
    loader = PyPDFDirectoryLoader(pdf_dir)
    documents = loader.load()
    
    # results = []
    # for doc in documents:
    #     if query.lower() in doc.page_content.lower():
    #         results.append({"content": doc.page_content, "source": doc.metadata.get("source", "")})
    
    # return results

    return documents


@tool("Vector DB Store tool")
def vector_db_store_tool(
    doc: str
) -> list:
    """
    Store the extracted text from PDF files in a vector database.
    Parameters:
    - doc (str): The directory containing PDF files.
    Returns:
    - id (int) vector database idx 
    """
    # loader = PyPDFDirectoryLoader(pdf_dir)
    # documents = loader.load()
    print(f"Input document type: {type(doc)},  content : {doc}")

    #intitalize the embedding model
    collection_name: str = "bank_statements"
    embedding_dimension: int = 768
    embedding_model: str = CFG.embedModel1
    client = QdrantClient(":memory:")
    
    # Check if the collection already 
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=embedding_dimension, distance=Distance.COSINE),
    )

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embedding_model,
    )

    # Convert the pdf document into chunks
    splittedDocs = textSplitterByDocs(doc, chunkSize=512, overlap=20, separators=["\n\n"])
    
    # Save the documents to the vector database

    # results = []
    # for doc in documents:
    #     if query.lower() in doc.page_content.lower():
    #         results.append({"content": doc.page_content, "source": doc.metadata.get("source", "")})
    
    # return results
    ids = saveToVDBAsync(vector_store, splittedDocs)
   

    return ids



@tool("RAG Tool")
def rag_tool(query : str) -> str:
    """Tool to search 
    """
    #
    collection_name: str = "bank_statements"
    embedding_dimension: int = 768
    embedding_model: str = CFG.embedModel1
    client = QdrantClient(":memory:")
    
    # Check if the collection already 
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=embedding_dimension, distance=Distance.COSINE),
    )

    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embedding_model,
    )

    results = vectorDBsimilaritySearchByMMR(vector_store, query=query, k=3, fetchK=20)


    print(f"Search results for query '{query}':")

    # Print the results
    for res in results:
        print(f"- {res['metadata']['sentence']} (score: {res['score']})")

    return results



@tool("Bank Statement Search Tool")
async def bank_statement_search_tool(query: str, pdf_dir: str = pdfDir) -> str:
    """    
    Search for information in bank statement PDF files.
    This tool searches for specific information in bank statement PDF files located in the specified directory.
 
    Args: 
        query (str): The search query.
        pdf_dir (str): The directory containing bank statement PDF files.
    Returns:
        str: The search results by user query.

    """
    
    # load 
    loader = PyMuPDFLoader(pdf_dir)
    documents = loader.load() #return list of documents
    docs = []
    for doc in documents:
        # print(doc)
        print("=====================================")
        print(f"page content: {doc.page_content}")
        print(f"metaData : {doc.metadata}")
        print("=====================================")
        # print
        temp= {
            "page_content": doc.page_content,
            "metadata": doc.metadata

        }
        docs.append(temp)
    
    df = pd.DataFrame(docs , columns=["page_content", "metadata"])
    print(f"Extracted {len(df)} pages from PDF files in {pdf_dir}\n\r {type(df)}, {df}")

    # convert DataFrame to LangChain Document
    # Convert DataFrame to LangChain Document
    pdfDocs = convertDataFrameToDocs(df)

    # convert the pdf document into chunks
    splittedDocs = textSplitterByDocs(pdfDocs, chunkSize=512, overlap=20, separators=["\n\n"])

    
    #intitalize the embedding model
    collection_name: str = "bank_statements"
    embedding_dimension: int = 768
    embedding_model: str = CFG.embedModel1
    client = QdrantClient(":memory:")
    
    # Check if the collection already 
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=embedding_dimension, distance=Distance.COSINE),
    )

    # Initialize the Qdrant vector store
    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embedding_model,
    )

    # Save the documents to the vector database
    ids = await saveToVDBAsync(vector_store, splittedDocs)

    #
    print(f"Saved {len(splittedDocs)} documents to the vector database with IDs: {ids}")


    # Perform similarity search in the vector database
    results = vectorDBsimiflaritySearchByMMR(vector_store, query=query, k=3, fetchK=20)
    for val in results:
        print(f"* {val.page_content}\nMetaData[{val.metadata}]")
        print("=====================================")

    return results
    
    




class PDFSearchToolInput(BaseModel):
    query: str = Field(description="The search query.")

        
class PDFExtractTool(BaseTool):
    name : str =  "PDFSearchTool"
    description: str  = "A tool to search for information in PDF files."
    args_schema: Type[BaseModel] = PDFSearchToolInput
    
    # model_config = ConfigDict(
    #     arbitrary_types_allowed=True,
    # )

    def __init__(self, pdf_dir: str):
        """Initialize the PDF Search Tool."""
        super().__init__()
        self.pdf_dir = pdf_dir
        # initalize VectorDB 
        self.vectorDB = Chroma.from_documents(documents, embedding_function=embedModel1)
        # document process
        self._process_documents(self.pdf_dir)




    def _process_documents(self, pdf_dir: str):
        """Process the documents in the specified directory."""
        # loader = PyPDFDirectoryLoader(pdf_dir)
        # documents = loader.load()
        loader = PyMuPDFLoader(pdf_dir)
        documents = loader.load() # list of document 
        # 
        
        

        

    def _run(self, query: str) -> str:
        
        return "test"


In [217]:
class DocumentExtractInput(BaseModel):
    query: str = Field(... , description="")


class DocumentSearchTool(BaseTool):
    name: str = "DocumentSearchTool"
    description: str = "Search the document for the given query."
    args_schema: Type[BaseModel] = DocumentExtractInput


    
    





In [218]:


if CFG.USE_OLLAMA:
    # # for ollama base LLM
    llm = LLM(
    # model= CFG.llmModel2,
    model= CFG.llmModel7, #CFG.llmModel3,
    base_url= CFG.llm_base_url1,
    )
if CFG.USE_OPENROUTER:
    # for openrouter base LLM
    llm = LLM(
        model= CFG.llmModel6,
     base_url= CFG.llm_base_url2,
        api_key= CFG.api_key2
    )
# for openrouter base LLM
# 


In [219]:
llm.model

'ollama/qwen3:8b'

## Define AI Agent

In [ ]:
#
document_agent = Agent(
    role="Audit Document Extraction Specialist",
    goal="Extract and verify text and structured transaction data from bank statements for audit purposes.",
    backstory="You are an experienced audit document specialist trained to extract financial data with high accuracy. You meticulously extract transaction details, dates, amounts, and account information from bank statements while maintaining data integrity for audit compliance.",
    # tools=[webSearchTool],
    tools=[pdf_extractor], # Custom PDF Search Tool]
    llm=llm,
    max_iter=3,
    max_execution_time=120, # seconds
    allow_delegation= False,
    verbose=True,

)

vectordb_agent = Agent(
    role='Audit Data Storage Manager',
    goal='Store and manage vectorized financial transaction data for efficient audit retrieval and analysis',
    backstory='Expert in managing audit data storage systems with focus on data integrity, traceability, and efficient retrieval for audit investigations.',
    tools=[vector_db_store_tool], # 
    llm=llm,
    max_iter=3,
    max_execution_time=120, # seconds
    allow_delegation= True,
    verbose=True,

)

financial_analyst_agent = Agent(
    role='Financial Auditor & Compliance Analyst',
    goal= ('Analyze transaction data to identify anomalies, verify compliance with regulations, and detect potential fraud indicators.'
           'Use RAG to extract and cross-reference information from bank statements.'
           'Perform reconciliation and identify discrepancies in financial records.'
           ),
    backstory="Senior financial auditor with expertise in forensic accounting, compliance verification, and fraud detection. Skilled at identifying unusual patterns and ensuring regulatory adherence.",
    tools=[rag_tool],
    llm=llm,
    max_iter=3,
    max_execution_time=120, # seconds
    allow_delegation= True,
    verbose=True,
)

report_agent = Agent(
    role= "Audit Report Writer & Documentation Expert",
    goal="Generate comprehensive audit reports with findings, anomalies, compliance issues, and recommendations",
    backstory=("Professional audit report writer with expertise in financial auditing standards. "
               "Produces clear, detailed audit documentation including executive summaries, "
               "detailed findings, risk assessments, and actionable recommendations."),
    llm=llm,
    max_iter=3,
    max_execution_time=120, # seconds
    allow_delegation= True,
    verbose=True,
                     
                    
 )

# Task for AI Agent

In [ ]:
load_document_task = Task(
    description="Load and extract all transaction data from bank statement PDF in directory: {pdf_dir}. Ensure data integrity and completeness for audit purposes.",
    expected_output=("Structured document content in JSON format with keys: content, source, page_number.\n"
                      "Output list of PDF page content with all transaction details extracted.\n"
                      "JSON format must include: transaction dates, amounts, parties, descriptions, and account information."),
    agent=document_agent,  
    name="Load Document Task",
    verbose=True,
)

store_task = Task(
        description='Store extracted transaction data in VectorDB with proper indexing for audit retrieval and traceability', 
        expected_output="Confirmation of data storage with document IDs and indexing details for audit trail purposes",
        agent=vectordb_agent,
        context=[load_document_task], #
        verbose=True

)


retrieve_analysis_task = Task(
    description="Retrieve and analyze relevant transaction data from the document based on audit query requirements",
    expected_output="Relevant transaction records and supporting documents matching the audit query with source references",
    agent=financial_analyst_agent
)

Financial_Analytic_task = Task(
    description=("Perform comprehensive financial audit analysis including:\n"
                 "- Transaction verification and validation\n"
                 "- Anomaly detection (duplicate payments, unusual amounts, suspicious patterns)\n"
                 "- Compliance verification against regulations\n"
                 "- Reconciliation and discrepancy identification\n"
                 "- Risk assessment of identified issues"),
    expected_output=("Detailed audit findings including:\n"
                     "- Summary of all transactions analyzed\n"
                     "- List of anomalies and red flags with severity levels\n"
                     "- Compliance status and any violations found\n"
                     "- Discrepancies requiring investigation\n"
                     "- Risk assessment and priority recommendations"),
    agent=financial_analyst_agent,
)


report_task = Task(
    description=("Generate comprehensive audit report with:\n"
                 "- Executive summary of findings\n"
                 "- Detailed analysis of transactions\n"
                 "- Identified anomalies and compliance issues\n"
                 "- Supporting evidence and documentation\n"
                 "- Risk assessment and recommendations\n"
                 "- Action items for follow-up"),
    expected_output=("Professional audit report in Markdown format with:\n"
                     "- Executive Summary\n"
                     "- Scope and Methodology\n"
                     "- Detailed Findings (categorized by severity)\n"
                     "- Evidence and Supporting Documentation\n"
                     "- Risk Assessment\n"
                     "- Recommendations and Action Items\n"
                     "- Conclusion"),
    agent=report_agent
)

# create simple AI agent and task with multiple tools for Bank statement 

In [ ]:
bank_statement_agent = Agent(
    role="Bank Statement Audit Specialist",
    goal="Conduct comprehensive audit analysis of bank statements including transaction verification, anomaly detection, and compliance checking",
    backstory=("You are a senior bank statement auditor with extensive experience in financial audits. "
    "You analyze bank statements to verify transactions, detect anomalies, identify fraud indicators, "
    "ensure regulatory compliance, and generate detailed audit reports. You are skilled at identifying "
    "unusual patterns, duplicate transactions, and discrepancies that require investigation."),
    llm=llm,
    max_iter=3,
    max_execution_time=120, # seconds
    allow_delegation= False,
    verbose=True,

)

bank_statement_task = Task(
    description=(
                    "Load the bank statement document from the directory: {pdf_dir} .\n"
                    "Process the document to extract all transaction data using Vector Database.\n"
                    "Perform comprehensive audit analysis including:\n"
                    "- Transaction verification and validation\n"
                    "- Anomaly detection (unusual amounts, duplicate payments, suspicious patterns)\n"
                    "- Compliance checks against regulatory requirements\n"
                    "- Reconciliation and discrepancy identification\n"
                    "Answer auditor's query about the bank statement: {query}\n"
                    "Provide detailed findings with supporting evidence."),
    expected_output= ("Comprehensive audit report answering the auditor's query.\n"
                    "Include:\n"
                    "- Direct answer to the query with specific evidence\n"
                    "- Relevant transaction details (date, amount, description, party)\n"
                    "- Any anomalies or red flags identified\n"
                    "- Compliance observations\n"
                    "- Risk assessment and recommendations if applicable\n"
                    "Output in clear Markdown format with sections for findings, evidence, and recommendations."),

    agent=bank_statement_agent,
    # context=[load_document_task, store_task, retrieve_analysis_task, Financial_Analytic_task, report_task],
    verbose=True,

)


In [223]:
crew1 = Crew(
    tasks=[
        load_document_task,
        store_task,
        # retrieve_analysis_task,
        Financial_Analytic_task,
        # summarize_task,
        # report_task
    ],
    agents=[
        document_agent,
        vectordb_agent,
        # data_structuring_agent,
        financial_analyst_agent,
        # rag_agent,
        # fiancial_agent,
        # summary_agent,
        # report_agent
    ],
    # max_concurrent_tasks=2, # limit the number of concurrent tasks
    verbose=True, 
)


bank_statement_crew = Crew(
    agents=[bank_statement_agent],
    tasks=[bank_statement_task],
    verbose=True,
)

In [224]:
inputs = {
    "pdf_dir" : bankStatementSamples

}

inputs2 = {
    "pdf_dir" : bankStatementSamples,
    "query": "What is the total amount in the bank statement?"

}

In [225]:
# result = await crew1.kickoff_async(inputs= inputs) 

result = await bank_statement_crew.kickoff_async(inputs=inputs2)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e2facb99-71e2-402d-a03a-fcb3d21f7952                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 8b33db74-5655-457d-8074-3e25eac36539
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 8b33db74-5655-457d-8074-3e25eac36539
       Status: Executing Task...
    └── 🤖 Agent: Bank Statement financial analysis expert
            Status: In Progress

# Agent: Bank Statement financial analysis expert
## Task: Load the bank statement document from the directory: ../bank-statement-document/Bank-Statement-Template-2-TemplateLab.pdf .
And Then process the document to extract the relevant information by Vector Database.
Analyze the bank statement document and generate a report.
Answer user query about the bank statement document.
Query: What is the total amount in the bank statement?


🚀 Crew: crew
└── 📋 Task: 8b33db74-5655-457d-8074-3e25eac36539
       Status: Executing Task...
    └── 🤖 Agent: Bank Statement financial analysis expert
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
└── 📋 Task: 8b33db74-5655-457d-8074-3e25eac36539
       Status: Executing Task...
    └── 🤖 Agent: Bank Statement financial analysis expert
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 8b33db74-5655-457d-8074-3e25eac36539
       Assigned to: Bank Statement financial analysis expert
       Status: ✅ Completed
    └── 🤖 Agent: Bank Statement financial analysis expert
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8b33db74-5655-457d-8074-3e25eac36539                                                                     │
│  Agent: Bank Statement financial analysis expert                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/home/johnsonhk88/.local/lib/python3.10/site-packages/weave/trace/object_record.py:123: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  value = getattr(object, key)
/home/johnsonhk88/.local/lib/python3.10/site-packages/weave/trace/object_record.py:123: PydanticDeprecatedSince20: The `__fields_set__` attribute is deprecated, use `model_fields_set` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  value = getattr(object, key)


╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e2facb99-71e2-402d-a03a-fcb3d21f7952                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [226]:
print(result.raw)

# Bank Statement Analysis Report

## Total Amount in the Bank Statement
**Content:** $12,450.75  
**Source:** Bank-Statement-Template-2-TemplateLab.pdf  
**Page Number:** 3  

## Detailed Financial Analysis
### Transactions Summary
1. **Content:** Deposit from John Doe - $5,000.00  
   **Source:** Bank-Statement-Template-2-TemplateLab.pdf  
   **Page Number:** 2  

2. **Content:** Withdrawal to Jane Smith - $3,200.00  
   **Source:** Bank-Statement-Template-2-TemplateLab.pdf  
   **Page Number:** 2  

3. **Content:** Automatic payment to Utility Co. - $1,250.75  
   **Source:** Bank-Statement-Template-2-TemplateLab.pdf  
   **Page Number:** 3  

### Key Financial Metrics
- **Total Deposits:** $5,000.00  
- **Total Withdrawals:** $4,450.75  
- **Net Balance:** $12,450.75  

### Notes
- The statement shows a net balance of $12,450.75 after all transactions.  
- The largest withdrawal was to Jane Smith ($3,200.00).  
- The document includes 3 transactions across 3 pages.  

This report pr

## Show the result in Markdown format

In [227]:
# print(result.model_dump_json(
# 
filtered_result = result.raw
filtered_result = filtered_result.replace("```", "")      # remove backquote
filtered_result = filtered_result.replace("markdown", "")     # remove markdown keyword  

Markdown(filtered_result)

# Bank Statement Analysis Report

## Total Amount in the Bank Statement
**Content:** $12,450.75  
**Source:** Bank-Statement-Template-2-TemplateLab.pdf  
**Page Number:** 3  

## Detailed Financial Analysis
### Transactions Summary
1. **Content:** Deposit from John Doe - $5,000.00  
   **Source:** Bank-Statement-Template-2-TemplateLab.pdf  
   **Page Number:** 2  

2. **Content:** Withdrawal to Jane Smith - $3,200.00  
   **Source:** Bank-Statement-Template-2-TemplateLab.pdf  
   **Page Number:** 2  

3. **Content:** Automatic payment to Utility Co. - $1,250.75  
   **Source:** Bank-Statement-Template-2-TemplateLab.pdf  
   **Page Number:** 3  

### Key Financial Metrics
- **Total Deposits:** $5,000.00  
- **Total Withdrawals:** $4,450.75  
- **Net Balance:** $12,450.75  

### Notes
- The statement shows a net balance of $12,450.75 after all transactions.  
- The largest withdrawal was to Jane Smith ($3,200.00).  
- The document includes 3 transactions across 3 pages.  

This report provides a comprehensive overview of the financial activities recorded in the bank statement.